# 서브쿼리(SUBQUERY)
---

서브쿼리란 하나의 SQL문안에 포함되어 있는 도 다른 SQL문을 말한다. 

＊ 메인쿼리가 서브쿼리를 포함하는 종속적인 관계 → 서브쿼리는 메인쿼리의 칼럼을 모두 사용할 수 있지만 반대의 경우는 아니다.

SQL문에서 서브쿼리가 사용 가능한 곳

＊ SELECT / FROM / WHERE / HAVING / ORDER BY / INSERT 문의 VALUES절 / UPDATE문의 SET절


### 주의할 점!
1. 서브쿼리를 괄호로 감싸서 사용한다.
2. 서브쿼리는 단일 행 또는 복수 행 비교 연산자와 함께 사용 가능하다.   
    → 단일 행 비교 연산자는 서브쿼리의 결과가 반드시 1건 이하이어야 하고 복수행 비교연산자는 서브쿼리의 결과 건수와 상관 없다.
    
3. 서브쿼리에서는 ORDER BY 를 사용하지 못한다. (메인쿼리의 마지막 문장에 위치해야 한다.)

---
### ☆ 동작하는 방식에 따른 서브쿼리 분류
#### 1. UN-CORRELATED(비연관) 서브쿼리 
- 서브쿼리가 메인쿼리 칼럼을 가지고 있지 않는 형태의 서브쿼리로 메인쿼리에 서브쿼리가 실행된 결과를 제공하기 위한 목적으로 주로 사용된다.


#### 2. CORRELATED(연관) 서브쿼리  
- 서브쿼리가 메인쿼리 칼럼을 가지고 있는 형태의 서브쿼리이다. 
- 일반적으로 메인 쿼리가 먼저 수행되어 읽혀진 데이터를 서브쿼리에서 조건이 맞는지 확인하고자 할 때 주로 사용된다.

In [ ]:
# 선수 자신이 속한 팀의 평균 키보다 작은 선수들의 정보를 출력하는 SQL문

SELECT T.TEAM_NAME팀명, M.PLAYER_NAME 선수명, M.POSITION 포지션, M.BACK_NO 백넘버, M.HEIGHT 키
FROM PLAYER M, TEAM T
WHERE M.TEAM_ID = T.TEAM_ID AND M.HEIGHT < (SELECT AVG(S.HEIGHT)
                                           FROM PLAYER S
                                           WHERE S.TEAM_ID = M.TEAM_ID
                                           AND S.HEIGHT IS NOT NULL
                                           GROUP BY S.TEAM_ID)
ORDER BY 선수명;

In [ ]:
# 항상 영관서브쿼리로 사용되는 EXISTS 서브쿼리 → 조건을 만족하는 건이 여러 건이어도 만족하는 1건만 찾으면 추가적인 탐색을 진행하지 않는다.
# 20120501~20120502 사이 경기의 경기장 ID와 경기장명을 출력하는 SQL

SELECT STADIUM_ID ID, STADIUM_NAME 경기장명
FROM STADIUM A
WHERE EXISTS (SELECT 1
             FROM SCHEDULE X
             WHERE X.STADIUM_ID = A.STADIUM_ID
             AND X.SCHE_DATE BETWEEN '20120501' AND '20120502');

---
### ★ 반환되는 데이터 형태에 따른 서브쿼리 분류
#### 1. SINGLE ROW 서브쿼리(단일행 서브쿼리) 
- 서브쿼리의 실행 결과가 항상 1건 이하인 서브쿼리를 의미한다. 단일 행 서브쿼리는 단일 행 비교 연산자와 함께 사용된다.
    - 단일 행 비교 연산자 : =, <, <=, >, >=, <>
- 서브쿼리의 결과 건수가 2건 이상을 반환하면 SQL문은 RUN TIME 오류가 발생한다.

In [ ]:
# 예시1. '정남일' 선수가 소속된 팀에 소속된 선수들의 정보를 출력하는 서브쿼리

SELECT PLAYER_NAME 선수명, POSITION 포지션, BACK_NO 백넘버
FROM PLAYER
WHERE TEAM_ID = (SELECT TEAM_ID
                FROM PLAYER
                WHERER PLAYER_NAME = '정남일')
ORDER BY PLAYER_NAME;

In [ ]:
# 예시 2. 키가 평균 이하인 선수들의 정보를 출력하는 문제(그룹함수를 사용한 서브쿼리)

SELECT PLAYER_NAME 선수명, POSITION 포지션, BACK_NO 백넘버
FROM PLAYER
WHERE HEIGHT <= (SELECT AVG(HEIGHT)
                FROM PLAYER)
ORDER BY PLAYER_NAME;

#### 2. MULTI ROW 서브쿼리(다중행 서브쿼리)
- 서브쿼리의 실행 결과가 여러 건인 서브쿼리를 의미한다. 다중 행 서브쿼리는 다중 행 비교 연산자와 함꼐 사용된다.
    - 다중 행 비교 연산자 : IN, ALL, AVY, SOME, EXISTS

In [ ]:
# '정현수' 선수가 소속되어있는 팀 정보를 출력하는 서브쿼리 
# 내가 찾고자 하는 선수와 동명이인 '정현수' 선수가 있을 경우! → 서브쿼리의 결과가 2개 이상의 행이 반환될 경우 MULTI ROW 연산자 활용 필요

SELECT REGION_NAME 연고지명, TEAM_NAME 팀명, E_TEAM_NAME 영문팀명
FROM TEAM
WHERE TEAM_ID IN (SELECT TEAM_ID
                FROM PLAYER
                WHERE PLAYER_NAME = '정현수')
ORDER BY TEAM_NAME;

#### 3. MULTI COLUMN 서브쿼리 (다중칼럼 서브쿼리)
- 서브쿼리의 실행 결과로 여러 칼럼을 반환한다. 메인쿼리의 조건절에 여러 칼럼을 동시에 비교할 수 있다.
- 서브쿼리와 메인쿼리에서 비교하고자 하는 칼럼 개수와 칼럼의 위치가 동일해야 한다.

In [ ]:
# 소속팀별 키가 가장 작은 사람들의 정보를 출력하는 문제

SELECT TEAM_ID 팀코드, PLAYER_NAME 선수명, POSITION 포지션, BACK_NO 백넘버, HEIGHT 키
FROM PLAYER
WHERE (TEAM_ID, HEIGHT) IN (SELECT TEAM_ID, MIN(HEIGHT)
                           FROM PLAYER
                           GROUP BY TEAM_ID)
ORDER BY TEAM_ID, PLAYER_NAME;
# 해당 SQL문은 같은 팀당 여러명의 선수가 반환될 수 있는데, 이는 여러명의 선수가 똑같이 가장 작은 키를 가졌을 경우이다.

---
## 그 밖에 위치에서 사용하는 서브쿼리
#### SELECT 절에서 사용하기 
＊ SCALAR SUBQUERY : 한 행, 할 칼럼만을 반환하는 서브쿼리 → 단일 행 서브쿼리이므로 2건 이상 반환되면 오류 발생

In [ ]:
# 선수 정보와 해당 선수가 속한 팀의 평균 키를 함꼐 출력하는 예제

SELECT PLAYER_NAME 선수명, HEIGHT 키, (SELECT AVG(HEIGHT)
                                  FROM PLAYER X
                                  WHERE X.TEAM_ID = P.TEAM_ID) 팀평균키
FROM PLAYER P

#### FROM 절에서 사용하기
＊ INLINE VIEW : 서브쿼리의 결과가 마치 실행 시에 동적으로 생선된 테이블인 것처럼 사용할 수 있다. (JOIN을 사용하는 것과 같음)   

        → VIEW : 실제 데이터를 가지고 있지 않지만 테이블 역할을 수행하기 때문에 가상 테이블이라고도 한다.(뒤쪽에서 추가 설명)

In [ ]:
# 선수들 중에서 포지션이 미드필더인 선수들의 소속팀명 및 선수 정보를 출력하는 예제

SELECT T.TEAM_NAME 팀명, P.PLAYER_NAME 선수명, P.BACK_NO 백넘버
FROM (SELECT TEAM_ID, PLAYER_NAME, BACK_NO
      FROM PLAYER
      WHERE POSITION = 'MF') P, TEAM T
WHERE P.TEAM_ID = T.TEAM_ID
ORDER BY 선수명;

＊TOP-N : 인라인 뷰에 먼저 정렬을 수행하고 정렬된 결과 중에서 일부 데이터를 추출하는 것

In [ ]:
# 키가 큰 순서대로 선수들을 내림차순 정렬한 후 ROWNUM 을 활용하여 5명만 출력하는 예제
# 마지막 다섯번째 선수와 키가 같은 선수가 있어도 추가로 출력되지 않고, 인라인뷰에서 수행된 정렬된 5번째 선수까지만 출력된다.

SELECT PLAYER_NAME 선수명, POSITION 포지션, BAXK_NO 백넘버, HEIGHT 키
FROM (SELECT PLAYER_NAME, POSITION, BACK_NO, HEIGHT
     FROM PLAYER
     WHERE HEIGHT IS NOT NULL
     ORDER BY HEIGHT DESC)
WHERE ROWNUM <= 5;

#### HAVING 절에서 사용하기

＊ HAVING절은 그룹함수와 함께 사용될 때 그룹핑된 결과에 대해 부가적인 조건을 주기 위해서 사용한다.

In [ ]:
# 평균 키가 삼성 블루윙즈팀의 평균키보다 작은 팀의 이름과 해당 팀의 평균키를 구하는 예제

SELECT P.TEAM_ID 팀코드, T.TEAM_NAME 팀명, AVG(P.HEIGHT) 평균키
FROM PLAYER P, TEAM T
WHERE P.TEAM_ID = T.TEAM_ID
GROUP BY P.TEAM_ID, T.TEAM_NAME
HAVING AVG(P.HEIGHT) < (SELECT AVG(HEIGHT)
                       FROM PLAYER
                       WHERE TEAM_ID = 'K02');

#### UPDATE문의 SET 절에서 사용하기

In [ ]:
#  TEAM테이블에 STADIUM_NAME을 추가하였을 때 TEAM테이블에 추가된 STADIUM_NAME의 값을 STADIUM 테이블을 이용하여 변경하는 예제

UPDATE TEAM A
SET A.STADIUM_NAME = (SELECT X.STADIUM_NAME
                     FROM STADIUM X
                     WHERE X.STADIUM_ID = A.STADIUM_ID);

#### INSERT문의 VALUES 절에서 사용하기

In [ ]:
# PLAYER 테이블에 새로운 선수를 삽입할 때 PLAYER_ID를 현재 사용중인 값에 1을 더한 값으로 넣는 예제

INSERT INTO PLAYER(PLAYER_ID, PLAYER_NAME, TEAM_ID)
VALUES ((SELECT TO_CHAR(MAX(TO_NUMBER(PLAYER_ID))+1)
         FROM PLAYER), '홍길동', 'K06');

---
## VIEW (VIRTUAL TABLE)

 - VIEW사용의 장점
 1. 독립성 : 테이블 구조가 변경되어도 뷰를 사용하는 응용프로그램은 변경하지 않아도 된다.
 2. 편리성 : 복잡한 질의를 뷰로 생성함으로써 관련 질의를 단순하게 작성할 수 있다.(자주 사용할 때 편리함)
 3. 보안성 : 급여정보와 같이 숨기고 싶은 정보가 존재한다면 뷰를 생성할 떄 해당 칼럼을 빼고 생성함으로써 정보를 감출 수 있다.


In [ ]:
 # VIEW 생성 예시

CREATE VIEW V_PLAYER_TEAM   
AS SELECT P.PLAYER_NAME, P.POSITION, P.BACK_NO, P.TEAM_ID, T.TEAM_NAME   
   FROM PLAYER P, TEAM T   
   WHERE P.TEAM_ID = T.TEAM_ID;

In [ ]:
# VIEW는 이미 생성된 VIEW를 기반으로 생성할 수도 있다.

CREATE VIEW V_PLAYER_TEAM_FILTER
AS SELECT PLAYER_NAME, POSITION, BACK_NO, TEAM_ID, TEAM_NAME   
   FROM V_PLAYER_TEAM
   WHERE POSITION IN ('GK','MF');

In [ ]:
# VIEW 활용 예시 (이름이 황으로 시작하는 선수를 출력)

SELECT PLAYER_NAME, POSITION, BACK_NO, TEAM_ID, TEAM_NAME  
FROM V_PLAYER_TEAM
WHERE PLAYER_NAME LIKE '황%';

In [ ]:
# VIEW 삭제 예시

DROP VIEW V_PLAYER_TEAM;
DROP VIEW V_PLAYER_TEAM_FILTER;